In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [2]:
__name__ = "RNN"

# Model

In [3]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [4]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [5]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [6]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [7]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400
8/8 [==============================] - 5s 56ms/step - loss: 1.6017 - categorical_accuracy: 0.2785
Epoch 2/400
8/8 [==============================] - 0s 39ms/step - loss: 1.5770 - categorical_accuracy: 0.3882
Epoch 3/400
8/8 [==============================] - 0s 40ms/step - loss: 1.5311 - categorical_accuracy: 0.3671
Epoch 4/400
8/8 [==============================] - 0s 52ms/step - loss: 1.4911 - categorical_accuracy: 0.3966
Epoch 5/400
8/8 [==============================] - 0s 62ms/step - loss: 1.4927 - categorical_accuracy: 0.4008
Epoch 6/400
8/8 [==============================] - 0s 54ms/step - loss: 1.4991 - categorical_accuracy: 0.4219
Epoch 7/400
8/8 [==============================] - 0s 55ms/step - loss: 1.4923 - categorical_accuracy: 0.4430
Epoch 8/400
8/8 [==============================] - 0s 52ms/step - loss: 1.4102 - categorical_accuracy: 0.3924
Epoch 9/400
8/8 [==============================] - 0s 53ms/step - loss: 1.3447 - categorical_accuracy: 0.4008
Epoch 10/4

In [8]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 5)                 8

In [9]:
model.save(f"{modelPath}/{__name__}/test.h5")